Imports:

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import time

Simple function i made for testing and manually constructing testing the test functions used

In [ ]:
xi, yi= 0
output = {}
rows, cols = (20, 20)
arr = [[0]*cols]*rows

for xi in range(0,20):
  for yi in range(0,20):
    x = xi-10
    y = yi-10
    arr[xi][yi] = (1.5 - x + x*y)**2 + (2.25 - x + (x*y)**2)**2 + (2.625 - x + (x*y)**3)**2
  print(arr[xi])


TypeError: ignored

Defining the Particle object

In [ ]:
#Particle object.
class Particle:

  #Initalisation function
  def __init__(self,x0,bounds):
    self.position=[]
    self.velocity=[]
    self.posBest = []
    self.val = -1
    self.valBest = -1

    #Initalising random velocity and position
    for i in range(0,2):
      self.velocity.append(random.uniform(-1,1))
      self.position.append(random.uniform(bounds[0],bounds[1]))

    #initialising the particles best position
    self.posBest = self.position

  #Evaluating the particle based on different functions
  def eval (self,func):

    #Converting to x and y to aid readability, and ease conversion from functions to code
    x = self.position[0]
    y = self.position[1]

    #Evaluation the position based on the selected function
    if(func == 0):    #Booth Function
          self.val = (x+(2*y)-7)**2 + ((2*x) + y - 5)**2

    elif(func == 1):  #Matays Function
          self.val = 0.26 * (x**2 + y**2) - 0.48*(x*y)

    elif(func == 2):  #Beale Function
          self.val = (1.5 - x + x*y)**2 + (2.25 - x + (x*y)**2)**2 + (2.625 - x + (x*y)**3)**2

    #Checkign for best value
    if self.val < self.valBest or self.valBest == -1:
      self.posBest = self.position
      self.valBest = self.val

  #Calculates new velocity
  def updVel(self,posTarget,w,c1,c2):

    #Velocity For each dimension(x,y)
    for i in range(0,2):
      #Random attribute
      r1=random.random()
      r2=random.random()

      #Adjusting velocity for the personal best of the particle
      vel_cog = r1*c1*(self.posBest[i]-self.position[i])

      #adjusting the velocity towards a selected best
      vel_soc = r2*c2*(posTarget[i]-self.position[i])

      #Combinging calculated velocities with a weight and its existing velocity
      self.velocity[i] = w*(self.velocity[i]+vel_cog+vel_soc)

  #Calculates the new position of the particle
  def updPos(self,bounds):
    #For each dimension (x,y)
    for i in range(0,2):
      #Adding the velocity to he current position
      self.position[i] = self.position[i]+self.velocity[i]

      #Ensuing the particle cant leave the bounds set
      if(self.position[i] > bounds[1]):
        self.position[i] = bounds[1]

      if(self.position[i] < bounds[0]):
        self.position[i] = bounds[0]


'Main functions of the program'

In [ ]:
#Returns the best position of a random subsection of 3 particles
def getBestPos(swarm,numParticles):
  partsub = []
  bestpos = []
  partsub = [0 for i in range(3)]
  bestval = -999

  #Randomly selecting particles
  partsub[0], partsub[1], partsub[2] = random.sample(range(0, numParticles), 3)

  #finding the best position identified by any of the particles
  for i in range(0, 2):
    if swarm[partsub[i]].valBest < bestval or bestval == -999:
      bestpos = swarm[partsub[i]].posBest
      bestval = swarm[partsub[i]].valBest
  return bestpos

#'Main' Function containing the algorithm
def Algorithm(x0,bounds,numParticles,Iterations,w,c1,c2,func):
  bestPos = []
  swarm = []
  bestVal = -100
  partVal = [] #used to tracking progress / Iteration and could be removed to increace speed

  #Creating particles and adding them to the swarm
  for i in range(0,numParticles):
    swarm.append(Particle(x0,bounds))

  iter = 0

  #for the number of iterations selected
  while iter < Iterations:
    #For each of the particles each round
    for p in range(0,numParticles):
      swarm[p].eval(func) #Evaluating the particle

      #Identifying the best position of all the particle
      #This is primarily for tracking progress and could be removed to improve speed
      if(swarm[p].valBest < bestVal or bestVal == -100):
        bestPos = list(swarm[p].position)
        bestVal = float(swarm[p].valBest)

    #updating the velocity and position of each particle
    for p2 in range(0,numParticles):
      swarm[p2].updVel(getBestPos(swarm,numParticles),w,c1,c2)
      swarm[p2].updPos(bounds)

    #tracking the best value found.
    partVal.append(bestVal)
    iter+=1

  return bestVal

After creation and defantition of the particle and the algorithm below is the 'Main' where the program iteself is ran with the varying problems


The section below was made for tuning the hyper parameters manually, and allowing for selection of the function etc in preperation for experimentation.

In [ ]:
#Defining constraints
init=[-10,10]
bounds=[-10,10]

#Defining Hyper Parameters
numParticles = 20
Iterations = 50

#Velocity Hyper Parameters
W = 0.2
C1 = 0.5
C2 = 0.5

#Function Parameter -- 0 = Booth Function ---1 = Matyas function --- 2 = Beale function
f = 0

#Running Algorothm
output = Algorithm(init,bounds,numParticles,Iterations,W,C1,C2,f)

print("best value found : " + str(output))

best value found : 0.0018974878726660655


In [ ]:
#Number of tests for each function
Tests = 10

#Defining output 2D array
rows, cols = (3, Tests)
output = [[0]*cols]*rows

for f in range(0,3): #iterating through functions
  timelst = []

  for t in range(0,Tests): #running the selected number of tests
    st = time.time()
    output[f][t]= Algorithm(init,bounds,numParticles,Iterations,W,C1,C2,f)
    et = time.time()
    timelst.append(et-st)

  print( "Function " + str(f) + " best value for each run:")
  print(output[f])
  print("Standard Deviation : " + str(np.std(output[f])))
  print("Average time/Run : " + str(np.mean(timelst)) + "s")
  print("")

Function 0 best value for each run:
[1.0357821971713428, 0.27679226921814576, 0.907984944979389, 2.0760870610403153, 0.33741193092462907, 0.22936279970229106, 0.37954030804853667, 0.012505285528111521, 0.005456850004823424, 0.07229777298958134]
Standard Deviation : 0.6128316140961644
Average time/Run : 0.012293815612792969s

Function 1 best value for each run:
[2.479932264545456e-05, 8.930183977067909e-05, 0.0015064889816416142, 0.0007889910445277098, 0.003825867279312979, 0.001472183429068635, 0.0006537015665348299, 0.0002212296009508406, 0.008914194834739805, 4.015799447595707e-05]
Standard Deviation : 0.002624720915018841
Average time/Run : 0.01152660846710205s

Function 2 best value for each run:
[2.813856606869185, 0.318851488892928, 1.1514338993151325, 1.811446923932883, 0.3563486815065574, 1.5582890288431959, 3.0204495312143917, 3.9490515718337313, 1.5676950745382467, 1.1837588972217052]
Standard Deviation : 1.1099297517144844
Average time/Run : 0.012755751609802246s

